# Python для анализа данных

использован блокнот: *Алла Тамбовцева, НИУ ВШЭ*

дополнения +++ : *Ян Пиле, НИУ ВШЭ*


### Автоматизация работы в браузере: библиотека `selenium`

Библиотека `selenium` – набор инструментов для интерактивной работы в браузере средствами Python. Вообще Selenium ‒ это целый проект, в котором есть разные инструменты. Мы рассмотрим один из самых распространенных ‒ Selenium WebDriver, модуль, который позволяет Python встраиваться в браузер и работать в нем как пользователь: кликать на ссылки и кнопки, заполнять формы, выбирать опции в меню и прочее. 

Мы будем использовать WebDriver для решения такой задачи: Необходимо выгрузить все адреса участковых избирательных  комиссий Ивановской области. 

Сначала загрузим веб-драйвер из библиотеки `selenium`. 

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

Если Python пишет `No module called selenium`, убедитесь, что у вас установлена эта библиотека. Самый надежный способ установить ее ‒ найти *Anaconda Command Prompt*, вписать строку `pip install selenium` и нажать *Enter*. Если *Anaconda Command Prompt* не находится, можно поступить так: запустить Jupyter Notebook, щелкнуть на черное окно консоли, нажать *Ctrl+Z* (остановить запуск Jupyter), а потом так же ввести в этом окне строку `pip install selenium` и нажать *Enter*.

Затем нужно выбрать браузер и открыть новое окно через Python. Для этого нужно вызвать функцию, которая отвечает за открытие браузера. Обычно используется Chrome.

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

Если код выше не исполняется, скачайте файл с веб-драйвером [отсюда](https://sites.google.com/a/chromium.org/chromedriver/downloads), распакуйте архив и пропишите путь к файлу в круглых скобках (в примере файл с расширением exe на Windows). Чтобы прописать путь на Mac'е или Linux'e, нужно выполнить такую команду в терминале:
    
    export PATH="$PATH:/Путь/до/драйвера"

У меня было так: 
    
    export PATH="$PATH:/Users/i.pile/Documents/chromedriver"


In [ ]:
# br = wb.Chrome('C:/Users/student/Desktop/chromewebdriver/chromedriver.exe')

In [ ]:
# br = wb.Chrome("/Users/allat/Downloads/chromedriver")

In [ ]:
from fake_useragent import UserAgent
UserAgent().chrome

In [ ]:
import requests
from bs4 import BeautifulSoup
url = "http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/"
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})

In [ ]:
BeautifulSoup(response.text)

In [ ]:
driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

Раньше эта ссылка работала иначе! За время этих лекций сайт успел несколько раз измениться, так что нам придется переконфигурировать нашу работу :)
У нас открывается страница, которая мешает просмотру (там что-то написано о персональных данных пользователей). Ее надо закрыть. Чтобы закрыть ее, нужно ткнуть в какой-то кликабельный элемент, прямо как если бы вы физически открывали эту страницу. Чтобы найти кликабельный элемент, нужно зайти в код страницы, навести на интересующий вас элемент и,например, скопировать его XPath. Что за  XPath, спросите вы. Это путь, по которому можно найти элемент в Xml- или html-размеченном документе. Скажем, в документе вида:

    <tag1>
        <tag2>
            <tag3>
            </tag3>
        </tag2>
    </tag1>

Путь до tag3 будет иметь вид:

    /tag1/tag2/tag3

In [ ]:
fld = driver.find_element_by_xpath('/html/body/div[2]/div/div/button')
fld.click()

Ура, страница открылась. Мы хотим искать по региону и номеру участка, стало быть нужно два поля. Сохраним номер участка в переменную `n_uik`, а регион ‒ в `reg`.

In [ ]:
n_uik = 244
reg = "Ивановская область"

Вопрос: как эти два поля заполнить? Нужно найти их на странице, открытой в браузере, и вписать туда нужные строки. Только сделать это нужно через Python. 

![](selenium1.png)

Надо найти этот элемент, как в прошлый раз:
Код элемента -> Скопировать XPath

![](selenium2.png)

Теперь туда надо нажать

In [ ]:
fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
fld.click()

Мы, наконец, попали на отображение функциональности с выбором поля УИК и Региона. Чтоб ввести регион, нужно в соответствующее место кликнуть. Мы такое уже делали:

In [ ]:
fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
fld.click()

В реальной жизни мы бы после нажали ENTER. Здесь тоже так можно (последняя команда)

In [ ]:
# вводим название региона в поле - метод send_keys
reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
reg_field.send_keys(reg)

In [ ]:
reg_field.send_keys(Keys.RETURN)

Теперь нужно выбрать номер участка. Для этого осталось ввести в поле нужный нам номер (пока что у нас 244)

![](selenium3.png)

Все, заталкиваем в это поле наш УИК и пытаемся искать :)

In [ ]:
# вводим номер УИКа в поле - метод send_keys
uik_field = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[2]/div/input')
uik_field.click()
uik_field.send_keys(n_uik)

In [ ]:
uik_field.send_keys(Keys.RETURN)

В браузере открылась страница с адресом избирательного участка. 

![](selenium4.png)

In [ ]:
driver.close()

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
from time import sleep

driver.get("http://www.cikrf.ru/services/lk_address/?do=find_by_uik")

fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
fld.click()

sleep(1.5)

fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
fld.click()

sleep(1.5)

fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
fld.click()

sleep(1.5)

reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
reg_field.send_keys(reg)

sleep(1.5)

reg_field.send_keys(Keys.RETURN)

sleep(1.5)

uik_field = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[2]/div/input')
uik_field.click()
uik_field.send_keys(n_uik)

sleep(1.5)

uik_field.send_keys(Keys.RETURN)

Осталось выгрузить адрес для голосования. 

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
my_page = BeautifulSoup(driver.page_source)

In [ ]:
my_page

In [ ]:
my_page.text

In [ ]:
[i.text for i in my_page.find_all('span', {'class':'address'})]

In [ ]:
p = filter(lambda x: x not in ('', None), [i.text for i in my_page.find_all('span',{'class':'address'})])

In [ ]:
p = next(p)

Получилось! Единственное, хорошо бы учесть случаи, когда адреса участка в таком виде на странице нет (такие случаи бывают: иногда страница создана не по шаблону, иногда указан адрес территориальной комиссии). Для этого нам понадобится условие. Добавим «развилку»: пусть Python пробует найти адрес через указанное регулярное выражение, а если не найдет, то ищет его с помощью другого регулярного выражения. 

In [ ]:
# if p is None or p=='':
#     p = 'Адреса не нашлось'
p

Теперь у нас есть +- универсальный код, который позволяет найти адрес избирательного участка по номеру. В следующий раз мы оформим этот код в функцию, чтобы можно было подставлять в нее любой номер и регион, и применять ее в цикле, итерируя по номерам участков.

In [ ]:
driver.close()